In [102]:
import pandas as pd
import csv
import sys, json
from sqlalchemy import create_engine
import psycopg2
import pymongo
import datetime
import numpy as np
np.random.seed(1)
import requests
from pprint import pprint
import matplotlib.pyplot as plt
from flask import jsonify
import company_domain
company_domain=company_domain.company_domain

# Extraction

##  Go to the files path to extract Inc 5000 Americas Fastest Growing Companies list

In [103]:
# The path to our CSV files to extract all Inc 5000 company list

# 2019 Inc 5000 company list
Inc_2019_csv = "Resources/inc5000-2019.csv"
# 2018 Inc 5000 company list
Inc_2018_csv = "Resources/inc5000-2018.csv"
# 2007-2017 Inc 5000 company list
Inc_10year_csv ="Resources/inc5000_all10years.csv"


### Read the files

In [104]:
# Read csv file
# Read 2019 Inc 5000 company list
Inc_2019_df = pd.read_csv(Inc_2019_csv)

# Read 2018 Inc 5000 company list
Inc_2018_df = pd.read_csv(Inc_2018_csv)


# Read 2007-2017 Inc 5000 company list
Inc_10year_df = pd.read_csv(Inc_10year_csv,encoding='cp1252')

## Scraping Financial Times Americas Fastest Growing Companies 2020

In [105]:
# Financial times url to scrape
ft_url = 'https://www.ft.com/americas-fastest-growing-companies-2020'

In [106]:
# read the html file for scarping tables 
ft_tables= pd.read_html(ft_url)

In [107]:
# check it's type
type(ft_tables)

list

In [108]:
# Check how many tables are there 
len(ft_tables)

1

In [109]:
# scrape the the first table 

ft_tables_df = ft_tables[0]

In [122]:
# Transforming scraped data

# Drop column we don't need
drop_ft_columns=ft_tables_df.drop(['Revenue 2018 [in $m]','Revenue 2015 [in $m]','Number of employees 2018'], axis=1)


# Remove unnecessary sign "*" from the first column 

drop_ft_columns['Name'] = drop_ft_columns['Name'].str.replace(r'*', '')

# rename columns 
rename_ft_df=drop_ft_columns.rename(columns={"Name":"Company_name","FT category":"Industry","Absolute growth rate [in %]":"Growth_rate_%","Founding Year":"Founding_year","Compound annual growth rate (CAGR) [in %]":"Compound_annual_growth_rate_%"})

# Add the year column 
rename_ft_df['Rank_year']= '2020'

rename_ft_df['Country']= 'United States'

# drop null values Growth

rename_ft_df=rename_ft_df.dropna(axis=1,how='all')

# remove duplicates and show clean data

clean_ft_df=rename_ft_df.drop_duplicates()
clean_ft_df.head()

,Rank,Company_name,Country,Industry,Growth_rate_%,Compound_annual_growth_rate_%,Founding_year,Rank_year
0,1,Niantic,United States,Games industry,180306.6,1117.4,2015,2020
1,2,UiPath,United States,Technology,37463.5,621.5,2005,2020
2,3,Publisher First (Freestar),United States,Advertising,36680.1,616.5,2015,2020
3,4,FreightWise,United States,Transport,30547.9,574.2,2015,2020
4,5,Veggie Noodle Co.,United States,Food & Beverage,24074.8,523.0,2015,2020


## API Call for Scraping Growjo top 1000 fastest growing companies in 2020.

In [125]:

base_url = "https://growjo.com/api?url="

ranking = []
estimated_revenues = []
job_openings = []
company_name = []
city = []
country = []
state = []
employees = []
founded = []
linkedin_url = []
url = []
industry = []
total_funding = []

for domain in company_domain:
    target_url = base_url + domain
    data = requests.get(target_url)
    try:
        response = data.json()
    except ValueError:
        pass
    ranking.append(response['ranking'])
    estimated_revenues.append(response['estimated_revenues'])
    company_name.append(response['company_name'])
    founded.append(response['founded'])
    city.append(response['city'])
    country.append(response['country'])
    state.append(response['state'])
    employees.append(response['employees'])
    url.append(response['url'])
    industry.append(response['industry'])
    total_funding.append(response['total_funding'])
    

In [126]:
growjo_df = pd.DataFrame({
    "Rank":ranking,
    "Revenue_$":estimated_revenues,
    "Company_name": company_name,
    "City":city,
    "Founding_year":founded,
    "Country":country,
    "State":state,
    "Number_of_employees":employees,
    "Industry":industry,
    "Total_funding":total_funding
})

# Remove duplicate values
growjo_df=growjo_df.drop_duplicates()
#growjo_df.head()

In [127]:
# Transforming Growjo top 1000 fastest growing companies in 2020
    
# Add the year column 
growjo_df['Rank_year']= '2020'


# Normalizing revenue in to mellions 
growjo_df['Revenue_$']= (growjo_df['Revenue_$']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))

# drop null values

growjo_df=growjo_df.dropna(axis=1,how='all')

# remove duplicates and show clean data

clean_growjo_df=growjo_df.drop_duplicates()
clean_growjo_df.head()

,Rank,Revenue_$,Company_name,City,Founding_year,Country,State,Number_of_employees,Industry,Total_funding,Rank_year
0,1.0,36.5 Million,LetsGetChecked,New York,2014.0,United States,NY,135,Hospital/Healthcare,$113M,2020
1,701.0,13.1 Million,100 Thieves,Los Angeles,2017.0,United States,CA,64,Entertainment,None,2020
2,832.0,97.3 Million,10X Genomics,Pleasanton,2012.0,United States,CA,628,Biotech,$242.6M,2020
3,124.0,29.9 Million,15Five,San Francisco,2011.0,United States,CA,206,Tech Services,$42.1M,2020
4,941.0,10.8 Million,2ULaundry,Charlotte,2015.0,United States,NC,43,Consumer,None,2020


# Transforming

### Cleaning 2019 Inc 5000 company list

In [128]:
# Drop column we don't need
drop_column_2019=Inc_2019_df.drop(['Profile','_ - previous_workers','_ - founded','_ - metro','url','_ - yrs_on_list'], axis=1)

# Add the year column 
drop_column_2019['Rank_year']= '2019'

# Rename columns 
rename_df_2019= drop_column_2019.rename(columns={"_ - rank":"Rank","name":"Company_name","state":"State","_ - revenue":"Revenue_$","_ - growth":"Growth_rate_%","_ - industry":"Industry","_ - workers":"Number_of_employees","city":"City"})


# re-Order the data to suitable format

cleaned_2019_df =rename_df_2019[['Rank','Rank_year','Company_name','Industry','Number_of_employees','Revenue_$','Growth_rate_%','City','State']]
#cleaned_2019_df.head()

### Cleaning 2018 Inc 5000 company list

In [117]:
# Drop column we don't need

drop_column_2018=Inc_2018_df.drop(['_ - id','_ - zipcode','_ - ifmid','_ - latitude','_ - longitude','_ - website','_ - state_l','_ - metrocode','_ - ifiid','_ - previous_workers','_ - metro','_ - founded','_ - url','_ - partner_lists - partner_lists','_ - yrs_on_list'], axis=1)

# Add the year column 
drop_column_2018['Rank_year']= '2018'

# Rename columns 
rename_df_2018=drop_column_2018.rename(columns={"_ - rank":"Rank","_ - company":"Company_name","_ - state_s":"State","_ - revenue":"Revenue_$","_ - growth":"Growth_rate_%","_ - industry":"Industry","_ - workers":"Number_of_employees","_ - state_l":"State","_ - city":"City"})
rename_df_2018.head()

# Normalizing revenue in to mellions 
rename_df_2018['Revenue_$']= (rename_df_2018['Revenue_$']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))


cleaned_2018_df =rename_df_2018[['Rank','Rank_year','Company_name','Industry','Number_of_employees','Revenue_$','Growth_rate_%','City','State']]

#cleaned_2018_df.head()


### Cleaning 10 year (2007-2017) Inc 5000 company list

In [118]:
# Rename columns 

drop_column_10year_df=Inc_10year_df.drop(['_ - state_l','_ - metro','_ - yrs_on_list'],axis=1)

# Add the year column 
rename_10year_df=drop_column_10year_df.rename(columns={"year":"Rank_year","_ - rank":"Rank","_ - company":"Company_name","_ - website":"Company_Website","_ - state_s":"State","_ - revenue":"Revenue_$","_ - growth":"Growth_rate_%","_ - industry":"Industry","_ - workers":"Number_of_employees","_ - founded":"Founded_Year","_ - city":"City"})


# Normalizing revenue in to mellions 
rename_10year_df['Revenue_$']= (rename_10year_df['Revenue_$']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))

cleaned_10year_df =rename_10year_df[['Rank','Rank_year','Company_name','Industry','Number_of_employees','Revenue_$','Growth_rate_%','City','State']]

# cleaned_10year_df.head()




#### Combine all Inc 5000 data in to a single dataframe

In [119]:
# combine Inc 5000 data in to a single dataframe
combine_data =[cleaned_2018_df,cleaned_2019_df,cleaned_10year_df]
Inc_5000_df= pd.concat(combine_data)

In [129]:
# Add the year column 
Inc_5000_df['Country']= 'United States'
Inc_5000_df.head()

,Rank,Rank_year,Company_name,Industry,Number_of_employees,Revenue_$,Growth_rate_%,City,State,Country
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI,United States
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO,United States
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL,United States
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO,United States
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ,United States


In [136]:
# Merge all data in to a single dataframe
merge_all =[Inc_5000_df,clean_growjo_df,clean_ft_df]
merged_df= pd.concat(merge_all)
merged_df.head()

# Add unique id for company name 
id_df=merged_df.groupby(['Company_name'], sort=True).ngroup().apply('{:006}'.format)

merged_df['ID']=id_df

merged_df.head()


,Rank,Rank_year,Company_name,Industry,Number_of_employees,Revenue_$,Growth_rate_%,City,State,Country,Founding_year,Total_funding,Compound_annual_growth_rate_%,ID
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI,United States,NaN,NaN,NaN,020280
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO,United States,NaN,NaN,NaN,016302
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL,United States,NaN,NaN,NaN,009836
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO,United States,NaN,NaN,NaN,022717
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ,United States,NaN,NaN,NaN,005863


In [142]:
# ready tables to load

Company=merged_df[['ID','Company_name','Number_of_employees','Industry','City','State','Country']]

Rank=merged_df[['ID','Rank','Rank_year']]

Years=merged_df[['ID','Founding_year','Rank_year']]

Growth=merged_df[['ID','Growth_rate_%','Compound_annual_growth_rate_%','Total_funding','Revenue_$']]


# Load

In [145]:
# Create database connection
engine = create_engine('postgresql+psycopg2://postgres:216724401@2@localhost:5432/top_business_db')
connection = engine.connect()


OperationalError: (psycopg2.OperationalError) FATAL:  database "top_companies_db" does not exist

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# confirm tables
engine.table_names()